In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pystan
import pandas as pd
import networkx as nx

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
# load nodes and edges from file
nodes = pd.read_csv('./data/got-s6-nodes.csv')
edges = pd.read_csv('./data/got-s6-edges.csv')

# initialize networkx graph object and add nodes and edges
G = nx.Graph(name='Game Of Thrones S06')
G.add_nodes_from(nodes['Id'])
edge_dict = [(x['Target'],x['Source'], {'weight' : x['Weight']}) for (_,x) in edges.iterrows()]
G.add_edges_from(edge_dict)

In [ ]:
# see http://edwardlib.org/tutorials/latent-space-models

model_code = """
data { 
    int<lower=2> N; // number of nodes
    int<lower=1> D; // dimension of latent space
    int<lower=0> X[N,N]; // adjacency matrix
} parameters { 
    matrix[N,D] z; // latent positions in space
    real alpha; // bias in the network
    real<lower=0> sigma; // std
} model {
    sigma ~ cauchy(0,10);
    alpha ~ normal(0,0.2);
    for (n in 1:N) z[n,:] ~ normal(0, sigma);
    for (i in 1:N){
        for (j in 1:N) {
            if (i == j) continue; // This should probably be improved
            X[i,j] ~ poisson(exp(alpha - distance(z[i,:], z[j,:]))); // another way to do the rate is to simply say poisson(1 / dist) instead
        }
    }
    
}
"""


In [ ]:
sm = pystan.StanModel(model_code=model_code)

In [ ]:
data = {'N' : len(G.nodes()), 'D' : 2, 'X' : nx.to_numpy_matrix(G, dtype=int)}

In [ ]:
sm.sampling(data=data, iter=100)